In [1]:
import sys
sys.path.insert(0, '/mnt/pure/bowmanlab/sizhang/pkgs/gvp/src')

from models import MQAModel
from util import load_checkpoint
import tensorflow as tf
import mdtraj as md
import numpy as np
import os
from glob import glob
from tensorflow import keras as keras
from tqdm import tqdm
from validate_performance_on_xtals import process_strucs, process_paths, predict_on_xtals
from optimal_threshold_protein_performance import determine_optimal_threshold, determine_recall_or_sensitivity

2025-03-10 10:07:37.671761: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-10 10:07:37.964952: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-10 10:07:38.591166: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/seas/lib64
2025-03-10 10:07:38.591276: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open s

In [2]:
git_data_dir = '/mnt/pure/bowmanlab/sizhang/pkgs/gvp/data/pm-dataset'

In [3]:
val_label_dictionary = np.load(f'{git_data_dir}/val_label_dictionary.npy',
                            allow_pickle=True).item()

# Load validation apo ids to select best epoch
val_set_apo_ids_with_chainids = np.load(f'{git_data_dir}/'
                                        'val_apo_ids_with_chainids.npy')

# Load test apo ids for evaluation
val_l_max = max([len(l) for l in val_label_dictionary.values()])

val_label_mask = np.zeros([len(val_label_dictionary.keys()), val_l_max], dtype=bool)
for i, l in enumerate(val_label_dictionary.values()):
    val_label_mask[i] = np.pad(l != 2, [[0, val_l_max - len(l)]])

val_true_labels = np.zeros([len(val_label_dictionary.keys()), val_l_max], dtype=int)
for i, l in enumerate(val_label_dictionary.values()):
    val_true_labels[i] = np.pad(l, [[0, val_l_max - len(l)]])

print('Balance in val set')
print(f'Total negative residues in val: {np.sum(val_true_labels[val_label_mask] == 0)}')
print(f'Total positive residues in val: {np.sum(val_true_labels[val_label_mask] == 1)}')

# Load test set input
test_label_dictionary = np.load(f'/{git_data_dir}/test_label_dictionary.npy',
                                allow_pickle=True).item()
test_set_apo_ids_with_chainids = np.load(f'{git_data_dir}/'
                                        'test_apo_ids_with_chainids.npy')

# Load test apo ids for evaluation
test_l_max = max([len(l) for l in test_label_dictionary.values()])

test_label_mask = np.zeros([len(test_label_dictionary.keys()), test_l_max], dtype=bool)
for i, l in enumerate(test_label_dictionary.values()):
    test_label_mask[i] = np.pad(l != 2, [[0, test_l_max - len(l)]])

test_true_labels = np.zeros([len(test_label_dictionary.keys()), test_l_max], dtype=int)
for i, l in enumerate(test_label_dictionary.values()):
    test_true_labels[i] = np.pad(l, [[0, test_l_max - len(l)]])

print('Balance in test set')
print(f'Total negative residues in test: {np.sum(test_true_labels[test_label_mask] == 0)}')
print(f'Total positive residues in test: {np.sum(test_true_labels[test_label_mask] == 1)}')


Balance in val set
Total negative residues in val: 375
Total positive residues in val: 436
Balance in test set
Total negative residues in test: 1283
Total positive residues in test: 563


In [4]:
# Create model
DROPOUT_RATE = 0.1
NUM_LAYERS = 4
HIDDEN_DIM = 100

# get NN directories
nn_dirs = [
    '/mnt/pure/bowmanlab/ameller/gvp/task2/train-with-4-residue-batches-no-balancing-intermediates-in-training/net_8-50_1-32_16-100_dr_0.1_nl_4_hd_100_lr_2e-05_b4resis_b1proteins_20epoch_feat_method_gp-to-nearest-resi-procedure_rank_7_stride_1_window_40_pos_20_refine_feat_method_fpocket_drug_scores_max_window_40_cutoff_0.3_stride_5'
]

In [5]:
X_val, S_val, mask_val = process_paths(val_set_apo_ids_with_chainids, use_tensors=False)
_, S_val_lm, _ = process_paths(val_set_apo_ids_with_chainids, use_tensors=False, use_lm=True)

X_test, S_test, mask_test = process_paths(test_set_apo_ids_with_chainids, use_tensors=False)
# print(np.sum(test_label_mask, axis=1))
# print(np.sum(mask_test, axis=1))

print([(p, s1, s2) for p, s1, s2 in
        zip(test_set_apo_ids_with_chainids, np.sum(mask_test.astype(bool) & test_label_mask, axis=1), np.sum(test_label_mask, axis=1))])
# print(np.sum(test_label_mask, axis=1))

/export/seas-home/sizhang/mambaforge/envs/pocketminer/lib/python3.10/site-packages/mdtraj/formats/pdb/pdbstructure.py:576: UserWarning: WARNING: two consecutive residues with same number (ATOM   2377  N   TYR A 184      58.638  67.513  16.505  1.00  0.00           N  , ATOM   2376  O   GLY A 184      59.058  66.303  14.675  1.00  0.00           O  )
  warnings.warn(
/export/seas-home/sizhang/mambaforge/envs/pocketminer/lib/python3.10/site-packages/mdtraj/formats/pdb/pdbstructure.py:576: UserWarning: WARNING: two consecutive residues with same number (ATOM   2446  N   LYS A 188      54.888  70.473  20.325  1.00  0.00           N  , ATOM   2445  O   GLY A 188      53.058  71.553  21.025  1.00  0.00           O  )
  warnings.warn(
/export/seas-home/sizhang/mambaforge/envs/pocketminer/lib/python3.10/site-packages/mdtraj/formats/pdb/pdbstructure.py:576: UserWarning: WARNING: two consecutive residues with same number (ATOM   2816  N   GLN A 221      60.488  72.053  23.425  1.00  0.00        

[('6hb0A', 15, 15), ('2laoA', 15, 15), ('1urpA', 13, 13), ('6ypkA', 24, 24), ('3ugkA', 42, 42), ('5g1mA', 19, 19), ('5nzmA', 27, 27), ('4v38A', 23, 23), ('2w9tA', 40, 40), ('2hq8B', 25, 25), ('4r72A', 24, 24), ('5za4A', 24, 24), ('2ceyA', 18, 18), ('1kmoA', 22, 22), ('5niaA', 27, 27), ('2fjyA', 19, 19), ('3p53A', 16, 16), ('1kx9B', 32, 32), ('1tvqA', 35, 35), ('2zkuB', 31, 31), ('3nx1A', 11, 11), ('4i92A', 19, 19), ('3qxwB', 20, 20), ('5h9aA', 22, 22), ('2fd7A', 46, 46), ('4tqlA', 215, 215), ('2alpA', 178, 178), ('1ammA', 148, 148), ('4ape', 102, 102), ('5uoj', 106, 106), ('3gz0', 51, 51), ('1btp', 111, 111), ('2oss', 80, 80), ('5zn5', 134, 134), ('5x8ua', 112, 112)]


In [6]:
for nn_dir in nn_dirs:
        # if sidechain is in name, need to use tensors
        if 'sidechain' in nn_dir:
            continue

        # determine number of compeleted epochs
        val_files = glob(f"{nn_dir}/val_pr_auc_*.npy")
        print(len(val_files))

        if len(val_files) == 0:
            continue

        # Determine network name
        index_filenames = glob(f"{nn_dir}/*.index")
        print(index_filenames)
        nn_id = os.path.basename(index_filenames[0]).split('_')[0]

        if "lm" in nn_dir:
            if 'squeeze' in nn_dir:
                model = MQAModel(node_features=(8, 50), edge_features=(1, 32),
                                 hidden_dim=(16, HIDDEN_DIM),
                                 num_layers=NUM_LAYERS, dropout=DROPOUT_RATE,
                                 use_lm=True, squeeze_lm=True)
            else:
                model = MQAModel(node_features=(8, 50), edge_features=(1, 32),
                                 hidden_dim=(16, HIDDEN_DIM),
                                 num_layers=NUM_LAYERS, dropout=DROPOUT_RATE,
                                 use_lm=True, squeeze_lm=False)
        elif "pretrain" in nn_dir:
            model = MQAModel(node_features=(8, 100), edge_features=(1,32),
                             hidden_dim=(16,100), dropout=1e-3)
        else:
            model = MQAModel(node_features=(8, 50), edge_features=(1, 32),
                             hidden_dim=(16, HIDDEN_DIM),
                             num_layers=NUM_LAYERS, dropout=DROPOUT_RATE)

        # Determine which network to use (i.e. epoch with best AUC)
        pr_auc = []
        auc = []
        auc_metric = keras.metrics.AUC(name='auc')
        pr_auc_metric = keras.metrics.AUC(curve='PR', name='pr_auc')

        val_optimal_thresholds = []

        for epoch in tqdm(range(len(val_files))):
        # for epoch in tqdm(range(3)):
            nn_path = f"{nn_dir}/{nn_id}_{str(epoch).zfill(3)}"
            if 'lm' in nn_dir:
                predictions = predict_on_xtals(model, nn_path, X_val, S_val_lm, mask_val)
            else:
                predictions = predict_on_xtals(model, nn_path, X_val, S_val, mask_val)

            y_pred = predictions[mask_val.astype(bool) & val_label_mask]
            y_true = val_true_labels[val_label_mask]

            auc_metric.update_state(y_true, y_pred)
            pr_auc_metric.update_state(y_true, y_pred)

            pr_auc.append(pr_auc_metric.result().numpy())
            auc.append(auc_metric.result().numpy())

            # determine optimal threshold for validation set
            val_optimal_thresholds.append(determine_optimal_threshold(y_pred, y_true))

            # reset AUC and PR-AUC
            pr_auc_metric.reset_state()
            auc_metric.reset_state()

        # print(val_optimal_thresholds)
        print(f'Validation ROC-AUCs: {auc}')
        best_epoch = np.argmax(auc)
        print(f'best epoch is {best_epoch}')
        nn_path = f"{nn_dir}/{nn_id}_{str(best_epoch).zfill(3)}"
        print(f'using network {nn_path}')

        # Make predictions on test set
        if 'lm' in nn_dir:
            predictions = predict_on_xtals(model, nn_path, X_test, S_test_lm, mask_test)
        else:
            predictions = predict_on_xtals(model, nn_path, X_test, S_test, mask_test)

        test_y_pred = predictions[mask_test.astype(bool) & test_label_mask]
        test_y_true = test_true_labels[test_label_mask]

        # np.save(f'{nn_dir}/test_y_pred.npy', test_y_pred)
        # np.save(f'{nn_dir}/test_y_true.npy', test_y_true)

        # lengths = np.sum(mask_test.astype(bool), axis=1)
        # print(lengths)
        # parsed_y_pred = [np.array(y[:l]) for y, l in zip(predictions, lengths)]
        # np.save(f'{nn_dir}/test_y_pred_parsed.npy', parsed_y_pred)

        auc_metric.update_state(test_y_true, test_y_pred)
        pr_auc_metric.update_state(test_y_true, test_y_pred)

        print(f'Test AUC: {auc_metric.result().numpy()}')
        print(f'Test PR-AUC: {pr_auc_metric.result().numpy()}')

        # np.save(f'{nn_dir}/test_auc.npy', auc_metric.result().numpy())
        # np.save(f'{nn_dir}/test_pr_auc.npy', pr_auc_metric.result().numpy())

        # # Determine protein-level performance
        # # optimal_threshold = determine_optimal_threshold(test_y_pred, test_y_true)
        # optimal_threshold = val_optimal_thresholds[best_epoch]
        # print(optimal_threshold)

        # # parse only those predictions for cryptic residues and negative residues
        # parsed_y_pred = [np.array(y)[m] for y, m in zip(predictions, test_label_mask)]
        # parsed_y_true = [y[m] for y, m in zip(test_true_labels, test_label_mask)]
        # print(parsed_y_pred[0])
        # print(parsed_y_true[0])

        # protein_performance = determine_recall_or_sensitivity(parsed_y_pred, parsed_y_true, optimal_threshold)

        # np.save(os.path.join(nn_dir, 'test_protein_performance.npy'), protein_performance)

        # print([(p, round(v, 3)) for p, v in zip(test_label_dictionary.keys(), protein_performance)])

20
['/mnt/pure/bowmanlab/ameller/gvp/task2/train-with-4-residue-batches-no-balancing-intermediates-in-training/net_8-50_1-32_16-100_dr_0.1_nl_4_hd_100_lr_2e-05_b4resis_b1proteins_20epoch_feat_method_gp-to-nearest-resi-procedure_rank_7_stride_1_window_40_pos_20_refine_feat_method_fpocket_drug_scores_max_window_40_cutoff_0.3_stride_5/1646754348_000.index', '/mnt/pure/bowmanlab/ameller/gvp/task2/train-with-4-residue-batches-no-balancing-intermediates-in-training/net_8-50_1-32_16-100_dr_0.1_nl_4_hd_100_lr_2e-05_b4resis_b1proteins_20epoch_feat_method_gp-to-nearest-resi-procedure_rank_7_stride_1_window_40_pos_20_refine_feat_method_fpocket_drug_scores_max_window_40_cutoff_0.3_stride_5/1646754348_001.index', '/mnt/pure/bowmanlab/ameller/gvp/task2/train-with-4-residue-batches-no-balancing-intermediates-in-training/net_8-50_1-32_16-100_dr_0.1_nl_4_hd_100_lr_2e-05_b4resis_b1proteins_20epoch_feat_method_gp-to-nearest-resi-procedure_rank_7_stride_1_window_40_pos_20_refine_feat_method_fpocket_drug_s

  0%|          | 0/20 [00:00<?, ?it/s]

CHECKPOINT RESTORED FROM /mnt/pure/bowmanlab/ameller/gvp/task2/train-with-4-residue-batches-no-balancing-intermediates-in-training/net_8-50_1-32_16-100_dr_0.1_nl_4_hd_100_lr_2e-05_b4resis_b1proteins_20epoch_feat_method_gp-to-nearest-resi-procedure_rank_7_stride_1_window_40_pos_20_refine_feat_method_fpocket_drug_scores_max_window_40_cutoff_0.3_stride_5/1646754348_000
Shapes: [26 394 4 3] [26 394] [26 394]


  5%|▌         | 1/20 [00:08<02:37,  8.27s/it]

CHECKPOINT RESTORED FROM /mnt/pure/bowmanlab/ameller/gvp/task2/train-with-4-residue-batches-no-balancing-intermediates-in-training/net_8-50_1-32_16-100_dr_0.1_nl_4_hd_100_lr_2e-05_b4resis_b1proteins_20epoch_feat_method_gp-to-nearest-resi-procedure_rank_7_stride_1_window_40_pos_20_refine_feat_method_fpocket_drug_scores_max_window_40_cutoff_0.3_stride_5/1646754348_001
Shapes: [26 394 4 3] [26 394] [26 394]


...

 95%|█████████▌| 19/20 [01:41<00:04,  4.89s/it]

CHECKPOINT RESTORED FROM /mnt/pure/bowmanlab/ameller/gvp/task2/train-with-4-residue-batches-no-balancing-intermediates-in-training/net_8-50_1-32_16-100_dr_0.1_nl_4_hd_100_lr_2e-05_b4resis_b1proteins_20epoch_feat_method_gp-to-nearest-resi-procedure_rank_7_stride_1_window_40_pos_20_refine_feat_method_fpocket_drug_scores_max_window_40_cutoff_0.3_stride_5/1646754348_019
Shapes: [26 394 4 3] [26 394] [26 394]


100%|██████████| 20/20 [01:46<00:00,  5.32s/it]

Validation ROC-AUCs: [0.850841, 0.89637005, 0.8425535, 0.8395382, 0.82503974, 0.8723028, 0.83928746, 0.81579506, 0.8135963, 0.8434465, 0.8273792, 0.82836396, 0.8074771, 0.852471, 0.80400306, 0.80379206, 0.84629965, 0.83099085, 0.8170795, 0.81923854]
best epoch is 1
using network /mnt/pure/bowmanlab/ameller/gvp/task2/train-with-4-residue-batches-no-balancing-intermediates-in-training/net_8-50_1-32_16-100_dr_0.1_nl_4_hd_100_lr_2e-05_b4resis_b1proteins_20epoch_feat_method_gp-to-nearest-resi-procedure_rank_7_stride_1_window_40_pos_20_refine_feat_method_fpocket_drug_scores_max_window_40_cutoff_0.3_stride_5/1646754348_001
CHECKPOINT RESTORED FROM /mnt/pure/bowmanlab/ameller/gvp/task2/train-with-4-residue-batches-no-balancing-intermediates-in-training/net_8-50_1-32_16-100_dr_0.1_nl_4_hd_100_lr_2e-05_b4resis_b1proteins_20epoch_feat_method_gp-to-nearest-resi-procedure_rank_7_stride_1_window_40_pos_20_refine_feat_method_fpocket_drug_scores_max_window_40_cutoff_0.3_stride_5/1646754348_001
Shapes:

Test AUC: 0.8667573928833008
Test PR-AUC: 0.8110293745994568
